# Gradient Boosting Machines

In [ ]:
# command line install, uncomment line below
# !conda install -y -c conda-forge xgboost

In [ ]:
#######################
# standard code block #
#######################

# see https://ipython.readthedocs.io/en/stable/interactive/magics.html
%pylab inline

# sets backend to render higher res images
%config InlineBackend.figure_formats = ['retina']

#######################
#       imports       #
#######################
import pandas as pd
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
import xgboost as xgb

sns.set_style("whitegrid")

### Conceptual Overview & Hyperparameter Tuning with xgboost

**Objectives**:
1. Understand the basic mathematical framework behind gradient boosted trees and the model's key hyperparameters
2. Understand the typical use cases for gradient boosting and some of the pros and cons
3. Build familiarity with the widely used xgboost package and practice selecting/tuning hyperparameters 

### Mathematical Framework

**First Pass (Regression)**:

Let's start by briefly returning to random forests. With the RF algorithm, we start with a very low bias but high variance **strong learner**, and reduce its variance by averaging across many relatively uncorrelated copies. Recall that we get these copies by injecting randomness into each base tree in the forest ensemble (taking bootstrapped samples of the training data, subsampling the feature columns for each tree split). The end result is a model that overfits less than a single decision tree due to variance reduction. See the diagram from [KDnuggets](https://www.kdnuggets.com/2017/10/random-forests-explained.html) below.

![](images/RFDiagram.jpg)

Boosting models are a flipped version of this paradigm. Instead of a strong learner, we start with a high bias but low variance **weak learner** and reduce its bias by training models that iteratively correct for the mistakes made by previous models. Building a sort of tower of weak models in this fashion makes for a strong ensemble that doesn't underfit, and the low variance of each base model helps insulate the ensemble from overfitting.

The most powerful variety of boosting in modern ML is **gradient boosted trees**, which we'll focus on. Our base learner is a simple decision tree with relatively small depth / few decision splits. We will build a final model by adding the results of many such trees together. The most intuitive problem setting to start with here is regression, with the usual sum of the squared errors (SSE) as the loss function we aim to minimize for data $X, y$. In each step of model building, we fit a base learner tree to the residuals of the current model's training predictions, adding it to the ensemble and gradually decreasing our training error. 

In (mostly) english, here's what we do:

* We'll train $k$ (hyperparameter) base learner decision trees with max depth $d$ (hyperparameter) step by step, adding each to our overall model.
* Each base tree is fit using our data $X$, but the target values are the residuals $y - model_{cur}$, where $model_{cur}$ is our overall model at that time step (sum of previous trees and initialization)  
* We initialize our model with the best constant prediction, $mean(y)$.   

As an algorithm:

1. Set $T_0 = mean(y)$
2. For $m = 1,...,k$:
    1. Set $r_{m-1} = y - (T_0 + \sum\limits_{j=1}^{m-1}T_j(X))$
    2. Fit max depth $d$ tree $T_m$ with features $X$, target $r_{m-1}$
  
Obtain final model: $F(X) = T_0 + T_1(X) + ... + T_k(X)$

![](images/RegressDiagram.png)

Not too complicated! But hold on, where does the *gradient* part of the name come in? And would we be able to adapt this model to a classification problem? As it turns out, the answer to both questions is the same. 

**Generalizing GBM to Differentiable Loss Functions**:

Yep, the **gradient** bit is closely related to the gradient descent algorithm that we know and love. It's also where the real power of the GBM framework becomes apparent. With the more general version of the algorithm that we'll see below, we can simply plug in a differentiable loss function and get an appropriate model.

To understand the general approach, consider the prediction values $F(X)$ that approximate the target $y$ for an arbitrary model $F$. Without putting any constraints on the function $F$, we could choose a differentiable loss function $L(y_{true},y_{pred})$ and define the model that minimizes it:

$$F_* = argmin_{F}L(y,F(X))$$

But since our loss function $L$ is differentiable, we can solve this problem by using iterative gradient descent on an element-wise basis:

$$F_{m+1}(x_i) = F_m(x_i) - \alpha\nabla{L(y_i,F_m(x_i))}$$ 

There's a clear problem here. All we're doing above is adjusting each predicted $y_i$ value until it gets increasingly close to the true $y_i$, meaning that we're memorizing the training data and overfitting. To not overfit, we have to introduce some constraint that forces generalization. In this case, instead of using the pure gradient, we'll model the gradient at each step with a regression tree $T_{m+1}$, fit on $X$ as features and the individual gradients $-\frac{\partial}{\partial{F_m(x_i)}}L(y_{i},F_m(x_i))$ as targets. Doing this, we can modify the gradient descent update rule so that it becomes:

$$F_{m+1}(x_i) = F_m(x_i) + T_{m+1}(x_i)$$

This looks exactly like the algorithm we used for regression boosting. It seems a bit fancier than what we had earlier, but really the only difference is that we've substituted gradient terms (**pseudo-residuals**) for the simple residuals we had originally. In fact, for SSE regression as above the gradient terms are equal to the residuals, so it's simply a special case of this general version of boosting. And with this single change we've expanded from being able to model least squares regression problems to being able to model for any differentiable loss function.    

Here is our generalized algorithm:

1. Choose loss function $L(y_{true},y_{pred})$
2. Initialize with best constant prediction i.e. $F_0 = argmin_{\gamma}\sum\limits_{i=1}^{N}L(y_i,\gamma)$
2. For $m = 1,...,k$:
    1. For $i = 1,...,n$: Set $r_{m-1,i} = -\frac{\partial{L(y_{i},F_{m-1}(x_i))}}{\partial{F_{m-1}(x_i)}}$
    2. Fit max depth $d$ tree $T_m$ with features $X$, target $r_{m-1}$
    3. Update $F_m(x) = F_{m-1}(x) + T_m(x)$
  
Obtain final model: $F(X) = T_0 + T_1(X) + ... + T_k(X)$

Note that we adapt to $k > 2$ class classification problems by training k of these models using [softmax loss](https://www.kdnuggets.com/2016/07/softmax-regression-related-logistic-regression.html). 

### Pros and Cons of GBMs

Boosted tree models are often the gold standard for tabular data in modern machine learning. They are well-equipped to model complex ground-truth functions without overfitting, adapt to your loss function of choice, and scale well to large datasets. They tend to do quite well with messy data: tree methods handle missing values naturally and can filter out the least useful input variables based on feature selection for splitting. That said, they are usually more sensitive to hyperparameter choice than random forests and can be a bit tricky and time-consuming to tune. They also lack the explicit interpretability of simpler models like linear/logistic regression.

In short, GBMs are a powerful tool to apply to a wide range of datasets (in terms of loss objective, size, variable types, and messiness). They are probably the closest thing to a catch-all and out-of-the-box modeling approach that currently exists. In a use case where you have at least a reasonable amount of data, want an optimally predictive model, can sacrifice some interpretability, and have the time/resources to tune multiple hyperparameters, you should definitely try a GBM model.   

![](images/xgbcalm.png)

**Summary**

**Pros**:

* Adaptable to wide range of problem settings (regression, classification, unusual or custom loss functions)
* Potential to be highly expressive (model compex, non-linear relationships)...
* But still flexible enough to accomodate complexity constraints/regularization  

*General tree benefits:*

* Feature scale invariant / natural handling of heterogenous features
* Automatic handling of missing/NA values 
* Automatic feature interaction
* Robust to input outliers and unhelpful input features (automatic feature selection)
* Scales well to large datasets

**Cons**:

* Many hyperparameters to tune for full power -- can be time-consuming and cause overfitting if not well-validated
* Lack clear/clean interpretability
* Tree methods struggle with extrapolation (can perform poorly on feature values outside of training range) 
* Can struggle with handling categorical variables (especially if large # of categories)

### Hands on GBMs and parameter tuning

Now that we've covered some of the theory behind GBMS and their pros and cons, let's turn to a concrete example of training one. We'll use [xgboost](https://github.com/dmlc/xgboost) (extreme gradient boosting), currently the most popular python package when it comes to training production-level boosting models. See [here](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn) for documentation on the sklearn-style interface for XGBoost that we'll be using.

Another popular option is Microsoft's [lightgbm](https://github.com/Microsoft/LightGBM) package, which is  faster and sometimes more accurate than xgboost, but typically harder to tune and closer to bleeding edge as software. Sci-kit learn does have an easier to use [gbm implementation](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html), but it's less efficient and less powerful.

Xgboost makes some adjustments to the algorithm described above and introduces a number of powerful efficiencies that allow the package to scale well. [This post](https://medium.com/@Synced/tree-boosting-with-xgboost-why-does-xgboost-win-every-machine-learning-competition-ca8034c0b283) is a good overview comparison of classic gradient boosting and xgboost, and the [xgboost paper](https://arxiv.org/pdf/1603.02754.pdf) digs into methodology and implementation details.

For this example we'll use the California housing dataset that we've seen before (regression on housing prices). We'll first look at a couple of benchmarks, then step through the process of tuning an xgboost model and see what it can do. The process would be essentially the same for a classification problem (part of what makes boosting so great), which we'll see in a quick example at the end on the Wisconsin breast cancer dataset.

In [ ]:
housing_dataset = fetch_california_housing()
X = pd.DataFrame(housing_dataset.data)
X.columns = housing_dataset.feature_names
y = housing_dataset.target

In [ ]:
X.describe()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

#Split data into 3: 60% train, 20% validation, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2018)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=2019)

#Evaluate models with Root Mean Squared Error
def rmse(actuals, preds):
    return np.sqrt(((actuals - preds) ** 2).mean())

Linear regression benchmark:

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)
rmse(lr.predict(X_test), y_test)

Random forest benchmark:

In [ ]:
rf = RandomForestRegressor(n_estimators = 100, max_features = 3, n_jobs=-1)
rf.fit(X_train,y_train)
rmse(rf.predict(X_test), y_test)

**Xgboost Tuning**

Now that we've set a solid baseline, let's turn to xgboost. Most of the parameters we'll use here should be familiar from the first section of this notebook or similar to what we've seen for random forests, with the exception of *learning rate*.

**n_estimators**: number of base learner trees  
**max_depth**: max depth per base tree (typical values are 3-12)   
**learning_rate**: shrinkage factor applied to each base tree update  
**subsample**: row subsampling rate (similar to RF)   
**min_child_weight**: roughly the minimum allowable child samples for a tree split to occur  
**colsample_bytree**: feature subsampling rate (similar to RF) 

The use of a learning rate/shrinkage factor is a form of regularization that can greatly reduce overfitting. It typically trades off with the n_estimators and depth parameters (raising these add complexity) -- lower learning rate  usually wants higher n_estimators, higher max depth usually wants lower learning rate etc. The two subsampling parameters and min_child_weight are also forms of regularization. These types of tradeoffs are part of why it typically works better to follow a manual tuning procedure than to try a massive grid search across different parameter combinations. That simply doesn't scale well to large datasets. 

Instead of trying to pick n_estimators by hand or do an intensive grid search, we'll use the **early stopping** technique as a massive time saver. This means that we track our validation error at each update step in the training process, and halt training once validation error hasn't improved in some number of **early_stopping_rounds**. This way, we can set n_estimators to a very large number and simply train until we land at a good stopping point for validation error. 

(One technical problem with early stopping is that n_estimators selection can overfit to the validation data. One reasonable solution is to combine early stopping with k-fold cross validation, and use the average n_estimators across k training rounds for training a new model).   

Here is our first xgboost example:

In [ ]:
gbm = xgb.XGBRegressor( 
                       n_estimators=30000, #arbitrary large number
                       max_depth=3,
                       objective="reg:squarederror",  # Other options: https://xgboost.readthedocs.io/en/latest/parameter.html#learning-task-parameters
                       learning_rate=.1, 
                       subsample=1,
                       min_child_weight=1,
                       colsample_bytree=.8
                      )

eval_set=[(X_train,y_train),(X_val,y_val)] #tracking train/validation error as we go
fit_model = gbm.fit( 
                    X_train, y_train, 
                    eval_set=eval_set,
                    eval_metric='rmse',
                    early_stopping_rounds=20,
                    verbose=True #gives output log as below
                   )

Starting xgb benchmark: we've already improved on RF.

In [ ]:
# predict using n_estimators with lowest validation error
rmse(gbm.predict(X_test, ntree_limit=gbm.best_ntree_limit),y_test) 

To tune, we should use validation results and ignore test until final verification. So here's the validation error benchmark we want to beat:

In [ ]:
rmse(gbm.predict(X_val, ntree_limit=gbm.best_ntree_limit),y_val) 

---
**Xgboost Exercise**

We'll approach tuning using this general framework, starting from the original parameters:

1. Tune max_depth
2. Tune subsample
3. Tune min_child_weight
4. Tune colsample_bytree
5. Set learning_rate to .05 and use early stopping to get n_estimators

There are even more [xgboost parameters](https://github.com/dmlc/xgboost/blob/master/doc/parameter.rst) such as regularization terms that are worth tuning, but this framework should be a starting point for solid parameters.

For this process we'll repeat the cell block below with each adjustment and track our progress with comments, and tune in this way: say for max_depth, starting from 3 we try 5, 7 etc. until we stop improving then narrow down the search space.

Start by _replacing the final parameters below_ with the basic parameters listed in the first run above, and see if you can recover parameters close to those referenced here.

---

In [ ]:
#Step by step RMSEs, with .1 learning rate:
#best max_depth: 7, .452
#best subsample: .8, .448
#best min_child_weight: 12, .446
#best colsample_bytree: .7, .444

gbm = xgb.XGBRegressor(
    n_estimators=30000,  #arbitrary large number
    max_depth=7,
    objective="reg:squarederror",
    learning_rate=.05,
    subsample=.8,
    min_child_weight=12,
    colsample_bytree=.7,
    n_jobs=-1,
    random_state=0)

eval_set = [(X_train, y_train),
            (X_val, y_val)]  #tracking train/validation error as we go
fit_model = gbm.fit(
    X_train,
    y_train,
    eval_set=eval_set,
    eval_metric='rmse',
    early_stopping_rounds=50,
    verbose=False)
rmse(gbm.predict(X_val, ntree_limit=gbm.best_ntree_limit), y_val)

And our test MSE as a result of this process, a ~10% reduction in RMSE from the random forest model:

In [ ]:
rmse(gbm.predict(X_test, ntree_limit=gbm.best_ntree_limit),y_test) 

---

**Challenge**

If you'd like to challenge yourself to get more proficient with xgboost, try tuning this model even more and beat our current score. Do some research into other xgboost parameters that you could try using. I'd recommend l1, l2, and gamma regularization terms. 

---

**Interpretation**

When trying to interpret a boosting model, we're faced with a similar challenge to that of interpreting a RF model. Many people would refer to it as a "black box" model. Luckily, there are some readily available ways to better understand feature importances and impacts on the model.  

The easiest method is very similar to RF feature importances. We can assign importance scores to features in a number of different ways, and xgboost lets us easily plot/extract these scores. The most common scoring methods are **frequency** (default, just calculates the # of total tree splits the feature is used in), and **gain** (probably better, calculates cumulative information gain from tree splits done on that feature).

As we can see in the charts below, the two methods tend to agree on which features are most important. However, gain views Median Income as much more important than any of the other features, which seems like a reasonable conclusion.  

In [ ]:
xgb.plot_importance(gbm)
xgb.plot_importance(gbm, importance_type='gain')

In [ ]:
gbm.get_booster().get_score(importance_type='weight') #extract raw frequency scores

In [ ]:
gbm.get_booster().get_score(importance_type='gain') #extract raw gain scores

A key problem with importance scores is that they only tell us which features are most relevant, not what the directional impact of the feature is on the model/target. Techniques have been developed to visually study these impacts that include [Partial Dependence](http://scikit-learn.org/stable/auto_examples/ensemble/plot_partial_dependence.html) and [Individual Conditional Expectation](https://arxiv.org/pdf/1309.6392.pdf) plots, but these are beyond the scope of this lecture.

**Classification Example**

Breast cancer dataset (binary classification). Boosting does not seem to work that well on this problem (unsuprising given a small sample size of <600). But this shows how we can apply xgboost to a classification problem by simply changing the objective and eval_metric parameters.

In [ ]:
bc_dataset = load_breast_cancer()
X = pd.DataFrame(bc_dataset.data)
X.columns = bc_dataset.feature_names
y = bc_dataset.target

#Split data into 3: 60% train, 20% validation, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=2)

In [ ]:
gbm = xgb.XGBClassifier( 
                        n_estimators=30000,
                        max_depth=4,
                        objective='binary:logistic', #new objective
                        learning_rate=.05, 
                        subsample=.8,
                        min_child_weight=3,
                        colsample_bytree=.8
                       )

eval_set=[(X_train,y_train),(X_val,y_val)]
fit_model = gbm.fit( 
                    X_train, y_train, 
                    eval_set=eval_set,
                    eval_metric='error', #new evaluation metric: classification error (could also use AUC, e.g.)
                    early_stopping_rounds=50,
                    verbose=False
                   )

accuracy_score(y_test, gbm.predict(X_test, ntree_limit=gbm.best_ntree_limit)) 